## Analyze an Re sweep for Specific End States
### V-Shape (V), Orbit (O), Single File: In-Line (SF), Headbutt: In-Line Anti (HB)
### Monitor group velocity, distance, and angle between swimmers

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 14 14:16:04 2020

@author: thomas
"""

#MODULES
import os,sys
import re
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
%matplotlib notebook
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator,AutoMinorLocator)
from scipy.signal import savgol_filter
import pathlib
from matplotlib import animation
from IPython.display import display, Image, HTML

mpl.rcParams['axes.linewidth'] = 1.5 #set the value globally

In [3]:
#CONSTANTS
cwd_PYTHON = os.getcwd()
PERIOD = 0.1
DT = 5.0e-3
RADIUSLARGE = 0.002
RADIUSSMALL = 0.001
FREQUENCY   = 10.0  

#Lists
#RLength
VReList=['0.5','0.6','0.7','0.8','0.9','1.0','2.0','3.0','4.0','5.0','5.5','6.0','6.5','7.0','7.5','10.0','12.5','15.0',
        '17.5','20.0','25.0','30.0','35.0','40.0','50.0','60.0']
OReList=['0.5','1.0','2.0','3.0','4.0','5.0','5.5','6.0','6.5','7.0','7.5','10.0','12.5','15.0',
        '17.5','20.0','25.0','30.0','35.0','40.0','50.0','60.0']
HBReList=['0.5','1.0','2.0','3.0','4.0','5.0','7.5','10.0','12.5','15.0',
        '17.5','20.0','25.0','30.0','35.0','40.0','50.0','60.0']
SFReList=['0.5','0.6','0.7','0.8','0.9','1.0','2.0','3.0','4.0','5.0','5.5','6.0','6.5','7.0','7.5','10.0','12.5','15.0']
VData = []
OData = []
HBData = []
SFData = []

#### Obtain Pos Data for all simulations
#### Limit time to first five oscillations

In [3]:
def StoreData(cwd_POSDATA):
    #global axAll
    #Reset position data every Sim
    pdData = []
    #Load position data
    pdData = pd.read_csv(cwd_POSDATA+'/pd.txt',delimiter=' ')
    #Save only every 20 rows (Every period)
    pdData = pdData.iloc[::20]
    #Reindex so index corresponds to period number
    pdData = pdData.reset_index(drop=True)
    #Print pdData to make sure it has been done properly
    #print(pdData.head(6))
    #Create Swimmer A and Swimmer B dataframes
    dict_A = {'xU':pdData['aXU'],'yU':pdData['aYU'],'xL':pdData['aXL'],'yL':pdData['aYL'],'time':pdData['time']}
    dataA  = pd.DataFrame(data=dict_A)
    dict_B = {'xU':pdData['bXU'],'yU':pdData['bYU'],'xL':pdData['bXL'],'yL':pdData['bYL'],'time':pdData['time']}
    dataB  = pd.DataFrame(data=dict_B)
    #print(dataA)
    #print(dataB)
    #Create a swimmer class for each
    swimA = Swimmer(dataA)
    swimB = Swimmer(dataB)
    simTime = swimA.time
    angleA = swimA.get_theta()
    angleB = swimB.get_theta()
    #print('Theta_A = ',angleA*180.0/np.pi)
    #print('Theta_B = ',angleB*180.0/np.pi)
    
    #Unused functions
    #Calculate Combined Swimmer Lab Angle
    #theta_comb = CalcCombinedLabAngle(swimA,swimB)
    #print('Lab Frame')
    #labHx, labHy = swimB.get_labH()
    
    
    #Calculate Combined CM
    xCM_comb, yCM_comb = CalcSystemCM(swimA.CM,swimB.CM)
    #Calculate the three parameters
    #Hx, Hy, and theta_BW in swimmer A's reference frame
    Hx, Hy, Hmag, theta_BW = swimB.CalcHxHyTheta(swimA)
    #Calculate the three parameters (L, Theta1, Theta2)
    LSS, theta1, theta2 = swimA.CalcLTheta1Theta2(swimB)
    
    
    #print('Lab Frame')
    #First, create a dataframe to contain this information
    dict_Param = {'Hx':Hx,'Hy':Hy,'Hmag':Hmag,
                  'Theta':theta_BW,'thetaA':angleA*180.0/np.pi,'thetaB':angleB*180.0/np.pi,
                  'xCM_comb':xCM_comb,'yCM_comb':yCM_comb,'time':simTime}
    parData = pd.DataFrame(data=dict_Param)
    #print(parData)
    
    #Calculate Velocity of combined CM
    parData['vx_comb'], parData['vy_comb'], parData['v_comb'] = 0.0, 0.0, 0.0
    for idx in range(1,len(parData['time'])):
        parData.loc[idx,'vx_comb'] = (parData.loc[idx,'xCM_comb'] - parData.loc[idx-1,'xCM_comb'])/(RADIUSLARGE*PERIOD*FREQUENCY)
        parData.loc[idx,'vy_comb'] = (parData.loc[idx,'yCM_comb'] - parData.loc[idx-1,'yCM_comb'])/(RADIUSLARGE*PERIOD*FREQUENCY)
    parData['v_comb'] = np.hypot(parData['vx_comb'],parData['vy_comb'])
    
    #Calculate Angular Velocity for swimmers A and B
    parData['wA'] = CalcAngularFrequency(parData['thetaA'])
    parData['wB'] = CalcAngularFrequency(parData['thetaB'])
    parData['wAVG'] = 0.5*(parData['wA']+parData['wB'])
    
    return parData


#### Find Angle Between Swimmers (Account for Quadrant)

In [4]:
def Rotate(xy, theta):
    # https://en.wikipedia.org/wiki/Rotation_matrix#In_two_dimensions
    #First Rotate based on Theta
    #Allocate Arrays
    rotationMatrix = np.zeros((2,2))
    #Calculate rotation matrix
    rotationMatrix[0,0] = np.cos(theta)
    rotationMatrix[0,1] = -1.0*np.sin(theta)
    rotationMatrix[1,0] = np.sin(theta)
    rotationMatrix[1,1] = np.cos(theta) 
    return rotationMatrix.dot(xy)

def Translate(xy, offset):
    return xy + offset

def CalcSystemCM(ACM,BCM):
    xCM_comb = 0.5*(ACM[0]+BCM[0])
    yCM_comb = 0.5*(ACM[1]+BCM[1])
    return (xCM_comb, yCM_comb)

def CalcCombinedLabAngle(A,B):
    norm_comb = np.zeros((2,len(A.xU)))
    UxCM = 0.5*(A.xU + B.xU)
    UyCM = 0.5*(A.yU + B.yU)
    LxCM = 0.5*(A.xL + B.xL)
    LyCM = 0.5*(A.yL + B.yL)
    LabX = UxCM - LxCM
    LabY = UyCM - LyCM
    Length = np.hypot(LabX,LabY)
    norm_comb[0,:] = LabX/Length
    norm_comb[1,:] = LabY/Length
    theta_comb = np.arctan2(norm_comb[1,:],norm_comb[0,:]) - np.pi/2.0 #Reference angle is when norm points straight up
    return (theta_comb)

def CalcAngularFrequency(theta):
    #Calculate Angular Velocity for swimmers A and B
    #Angles are in range [0,360). Find using periodic conditions for 0 and 360
    w = np.zeros(len(theta))
    for idx in range(1,len(w)):
        diff1 = theta[idx] - theta[idx-1]
        if(diff1 <= -180.0):
            diff = diff1 + 360.0
        elif(diff1 >= 180.0):
            diff = diff1 - 360.0
        else:
            diff = diff1
        w[idx] = diff/PERIOD
    return w
    

class Swimmer:
    def __init__(self,data):
        self.data  = data
        self.xU, self.yU = data['xU'], data['yU']
        self.xL, self.yL = data['xL'], data['yL']
        self.time = data['time']/2.0
        self.CM = np.zeros((2,len(self.time)))
        self.theta = np.zeros(len(self.time))
        self.normY = np.zeros((2,len(self.time)))
        self.normX = np.zeros((2,len(self.time)))
        self.labH = np.zeros((2,len(self.time)))
        self.Hx = np.zeros(len(self.time))
        self.Hy = np.zeros(len(self.time))
        self.rot_norm = np.zeros((2,len(self.time)))
        self.theta_BW = np.zeros(len(self.time))
        self.CalcCM()
        #print('CM calculated')
        self.CalcLabAngle()
        #print('theta Calculated')
        #print('theta = ',self.theta)
    
    def get_theta(self):
        return self.theta

    def get_labH(self):
        return (self.labHx/RADIUSLARGE, self.labHy/RADIUSLARGE)
    
    def get_norms(self):
        return (self.normX, self.normY)
        
    def CalcCM(self):
        self.xCM = 0.8*self.xU + 0.2*self.xL
        self.yCM = 0.8*self.yU + 0.2*self.yL
        self.CM[0] = self.xCM
        self.CM[1] = self.yCM
        #print('CM = ',self.CM)
        
    def CalcLabAngle(self):
        #Find swimming axis (normal y-axis)
        self.labX   = self.xU - self.xL
        self.labY   = self.yU - self.yL
        self.length = np.hypot(self.labX,self.labY)
        self.normY[0] = self.labX/self.length
        self.normY[1] = self.labY/self.length
        #self.normY  = np.array([self.labX/self.length,self.labY,self.length])
        #2) Calculate Theta
        for idx in range(len(self.time)):
            if(self.normY[1,idx] >= 0.0):
                self.theta[idx] = np.arccos(self.normY[0,idx])
                #Theta_a[idx] = np.arccos(norm_aX[idx])
            else:
                self.theta[idx] = -1.0*np.arccos(self.normY[0,idx])+2.0*np.pi
                #Theta_a[idx] = -1.0*np.arccos(norm_aX[idx])+2.0*np.pi
        #Calculate perpendicular axis (swimmer's x-axis)
        self.normX[0] = np.cos(self.theta - 0.5*np.pi)
        self.normX[1] = np.sin(self.theta - 0.5*np.pi)
        
    def CalcHxHyTheta(self,swimmerA):
        #Here, we will calculate three parameters
        #1) Hx: In the swimmer A's reference frame
        #2) Hy: In the swimmer A's reference frame
        #3) Theta_BW: Angle between swimmer A and swimmerB (w/ respect to swimmer A y-axis)
        #But first, we must calculate in lab frame for some
        #print('yCM = ',self.yCM)
        #print('A_yCM = ',swimmerA.yCM)
        self.labHx = self.xCM - swimmerA.xCM
        self.labHy = self.yCM - swimmerA.yCM
        self.Hmag  = np.hypot(self.labHx,self.labHy)
        self.labH[0], self.labH[1] = self.labHx, self.labHy
        #1) Hx: In swimmer A's reference frame
        #2) Hy: In swimmer A's reference frame
        self.Projection(swimmerA.normX,swimmerA.normY,swimmerA.CM)
        
        #3) Rotate A and B ccw by 2pi - Theta_a
        self.rotateAngle = 2.0*np.pi - swimmerA.theta
        #RotationMatrix (def RotationMatrix)
        #Create Swimmer Position Arrays ??
        #self.normY
        
        for idx in range(len(self.length)):
            self.rot_norm[:,idx] = Rotate(self.normY[:,idx],self.rotateAngle[idx])
            swimmerA.rot_norm[:,idx] = Rotate(swimmerA.normY[:,idx],self.rotateAngle[idx])
            if(self.rot_norm[1,idx] >= 0.0):
                self.theta_BW[idx] = np.arccos(swimmerA.rot_norm[:,idx].dot(self.rot_norm[:,idx]))%(2.0*np.pi)
            else:
                self.theta_BW[idx] = (-1.0*np.arccos(swimmerA.rot_norm[:,idx].dot(self.rot_norm[:,idx]))+2.0*np.pi)%(2.0*np.pi)
        
        return (self.Hx/RADIUSLARGE, self.Hy/RADIUSLARGE, np.hypot(self.Hx,self.Hy)/RADIUSLARGE, 180.0*self.theta_BW/np.pi)

    def CalcLTheta1Theta2(self,swimmerB):
        #Here, we will calculate three parameters
        #1) LSS: Length between small spheres. We will find LSSx and LSSy norms for angle calculations
        #2) Theta1: Angle between LSS and swim1 axis (inside angle)
        #3) Theta2: Angle between LSS and swim2 axis (inside angle)
        
        #1) Find LSSx and LSSy
        #Calc LSS, normLSSx, and normLSSy
        
        self.LSSx = swimmerB.xL - self.xL
        self.LSSy = swimmerB.yL - self.yL
        self.LSS  = np.hypot(self.LSSx,self.LSSy)
        self.normLSS[0] = self.LSSx/self.LSS
        self.normLSS[1] = self.LSSy/self.LSS
        # Calculate thetaLSS
        for idx in range(len(self.time)):
            if(self.normLSS[1,idx] >= 0.0):
                self.thetaLSS[idx] = np.arccos(self.normLSS[0,idx])
                #Theta_a[idx] = np.arccos(norm_aX[idx])
            else:
                self.thetaLSS[idx] = -1.0*np.arccos(self.normLSS[0,idx])+2.0*np.pi
        
        #2) Find Theta1: Angle between LSS and swim1 axis
        self.rotateAngleLSS = 2.0*np.pi - self.thetaLSS
        for idx in range(len(self.LSS)):
            self.rot_normLSS[:,idx] = Rotate(self.normLSS[:,idx],self.rotateAngleLSS[idx])
            self.rot_normLTT[:,idx] = Rotate(self.normY[:,idx],self.rotateAngleLSS[idx])
            swimmerB.rot_normLTT[:,idx] = Rotate(swimmerB.normY[:,idx],self.rotateAngleLSS[idx])

            if(self.rot_normLTT[1,idx] >= 0.0):
                self.theta1[idx] = np.arccos(self.rot_normLSS[:,idx].dot(self.rot_normLTT[:,idx]))%(2.0*np.pi)
                if(swimmerB.rot_normLTT[1,idx] >= 0.0):
                    self.theta2[idx] = np.arccos(-1.0*self.rot_normLSS[:,idx].dot(swimmerB.rot_normLTT[:,idx]))%(2.0*np.pi)
                else:
                    self.theta2[idx] = (-1.0*np.arccos(-1.0*self.rot_normLSS[:,idx].dot(swimmerB.rot_normLTT[:,idx]))+2.0*np.pi)%(2.0*np.pi)

            else:
                self.theta1[idx] = (-1.0*np.arccos(self.rot_normLSS[:,idx].dot(self.rot_normLTT[:,idx]))+2.0*np.pi)%(2.0*np.pi)
            
                if(swimmerB.rot_normLTT[1,idx] >= 0.0):
                    self.theta2[idx] = np.arccos(-1.0*self.rot_normLSS[:,idx].dot(swimmerB.rot_normLTT[:,idx]))%(2.0*np.pi)
                else:
                    self.theta2[idx] = (-1.0*np.arccos(-1.0*self.rot_normLSS[:,idx].dot(swimmerB.rot_normLTT[:,idx]))+2.0*np.pi)%(2.0*np.pi)
                self.theta1[idx] = 2.0*np.pi - self.theta1[idx]
                self.theta2[idx] = 2.0*np.pi - self.theta2[idx]
        
        return (self.LSS/RADIUSLARGE, self.theta1, self.theta2)
    
    def Projection(self,normX,normY,A_CM):
        #Here we find the distance component for swimmer A's axes
        #print('normY = ',normY)
        #print('CM = ',self.CM)
        for idx in range(len(self.time)):
            '''print('normY[:,idx] = ',normY[:,idx])
            print('CM[idx] = ',self.CM[:,idx])
            dot = np.dot(normY[:,idx],self.CM[:,idx])
            print('dot = ',dot)
            ycoord = dot*normY[:,idx]
            print('ycoord = ',ycoord)'''
            self.Hx[idx] = np.dot(normX[:,idx],(self.CM[:,idx]-A_CM[:,idx]))
            self.Hy[idx] = np.dot(normY[:,idx],(self.CM[:,idx]-A_CM[:,idx]))


#### Create dataframe of all pos data for all time for all sims

In [18]:
#The main goal of this script is to create an H-Theta Phase Space of all
#simulations for every period elapsed.
#Example: 20s of sim time = 200 periods. 200 H-Theta Plots
#We may find that we can combine the H-Theta data after steady state has been reached
#1) For each simulation, store position data for every period (nothing in between)
#2) Calculate separation distance between large spheres (H)
#3) Calculate relative heading (Theta)
#4) Calculate deltaH and deltaTheta (change after one Period)
#5) Plot H vs Theta (Polar) for each period

#In-Tandem V-Shape (V)
nRe = len(VReList)

count = 0
dict_allData = {'Re':[],'Hx':[],'Hy':[],'Hmag':[],'Theta':[],
                'vx_comb':[],'vy_comb':[],'v_comb':[],'wA':[],'wB':[],'wAVG':[],'time':[]}
VData = pd.DataFrame(data=dict_allData)
for Re in VReList:
    cwd_POSDATA = cwd_PYTHON+'/../PosData/V/Re'+Re+'/'
    strPrint = 'Re:'+Re
    print(strPrint)
    #Get All sim data and store
    simData = StoreData(cwd_POSDATA)
    #Add Simulation Parameter identifiers
    simData['Re'] = float(Re)
    dict_sim = {'Hx':simData['Hx'],'Hy':simData['Hy'],'Hmag':simData['Hmag'],'Theta':simData['Theta'],
                'vx_comb':simData['vx_comb'],'vy_comb':simData['vy_comb'],'v_comb':simData['v_comb'],
                'wA':simData['wA'],'wB':simData['wB'],'wAVG':simData['wAVG'],
                'Re':simData['Re'],'time':simData['time']}
    df_sim = pd.DataFrame(data=dict_sim)
    VData = pd.concat([VData,df_sim],ignore_index=True)
    count += 1
    #print('Re ='+Re)
VData = VData.sort_values(by=['Re','time'])
VData.to_csv(cwd_PYTHON+'/VData.csv',index=False,sep=' ')
print(len(VData['time']))
print('count = ',count)
print('Storing VData is complete! Onto Analysis')

#Orbit (O)
nRe = len(VReList)

count = 0
dict_allData = {'Re':[],'Hx':[],'Hy':[],'Hmag':[],'Theta':[],
                'vx_comb':[],'vy_comb':[],'v_comb':[],'wA':[],'wB':[],'wAVG':[],'time':[]}
OData = pd.DataFrame(data=dict_allData)
for Re in OReList:
    cwd_POSDATA = cwd_PYTHON+'/../PosData/O/Re'+Re+'/'
    strPrint = 'Re:'+Re
    print(strPrint)
    #Get All sim data and store
    simData = StoreData(cwd_POSDATA)
    #Add Simulation Parameter identifiers
    simData['Re'] = float(Re)
    dict_sim = {'Hx':simData['Hx'],'Hy':simData['Hy'],'Hmag':simData['Hmag'],'Theta':simData['Theta'],
                'vx_comb':simData['vx_comb'],'vy_comb':simData['vy_comb'],'v_comb':simData['v_comb'],
                'wA':simData['wA'],'wB':simData['wB'],'wAVG':simData['wAVG'],
                'Re':simData['Re'],'time':simData['time']}
    df_sim = pd.DataFrame(data=dict_sim)
    OData = pd.concat([OData,df_sim],ignore_index=True)
    count += 1
    #print('Re ='+Re)
OData = OData.sort_values(by=['Re','time'])
OData.to_csv(cwd_PYTHON+'/OData.csv',index=False,sep=' ')
print(len(OData['time']))
print('count = ',count)
print('Storing OData is complete! Onto Analysis')

#SingleFile: In-Line (SF)
nRe = len(VReList)

count = 0
dict_allData = {'Re':[],'Hx':[],'Hy':[],'Hmag':[],'Theta':[],
                'vx_comb':[],'vy_comb':[],'v_comb':[],'wA':[],'wB':[],'wAVG':[],'time':[]}
SFData = pd.DataFrame(data=dict_allData)
for Re in SFReList:
    cwd_POSDATA = cwd_PYTHON+'/../PosData/SF/Re'+Re+'/'
    strPrint = 'Re:'+Re
    print(strPrint)
    #Get All sim data and store
    simData = StoreData(cwd_POSDATA)
    #Add Simulation Parameter identifiers
    simData['Re'] = float(Re)
    simData['time'] = 2.0*simData['time']
    dict_sim = {'Hx':simData['Hx'],'Hy':simData['Hy'],'Hmag':simData['Hmag'],'Theta':simData['Theta'],
                'vx_comb':simData['vx_comb'],'vy_comb':simData['vy_comb'],'v_comb':simData['v_comb'],
                'wA':simData['wA'],'wB':simData['wB'],'wAVG':simData['wAVG'],
                'Re':simData['Re'],'time':simData['time']}
    df_sim = pd.DataFrame(data=dict_sim)
    SFData = pd.concat([SFData,df_sim],ignore_index=True)
    count += 1
    #print('Re ='+Re)
SFData = SFData.sort_values(by=['Re','time'])
SFData.to_csv(cwd_PYTHON+'/SFData.csv',index=False,sep=' ')
print(len(SFData['time']))
print('count = ',count)
print('Storing SFData is complete! Onto Analysis')

#Headbutt: In-Line Anti (HB)
nRe = len(VReList)

count = 0
dict_allData = {'Re':[],'Hx':[],'Hy':[],'Hmag':[],'Theta':[],
                'vx_comb':[],'vy_comb':[],'v_comb':[],'wA':[],'wB':[],'wAVG':[],'time':[]}
HBData = pd.DataFrame(data=dict_allData)
for Re in HBReList:
    cwd_POSDATA = cwd_PYTHON+'/../PosData/HB/Re'+Re+'/'
    strPrint = 'Re:'+Re
    print(strPrint)
    #Get All sim data and store
    simData = StoreData(cwd_POSDATA)
    #Add Simulation Parameter identifiers
    simData['Re'] = float(Re)
    simData['time'] = 2.0*simData['time']
    dict_sim = {'Hx':simData['Hx'],'Hy':simData['Hy'],'Hmag':simData['Hmag'],'Theta':simData['Theta'],
                'vx_comb':simData['vx_comb'],'vy_comb':simData['vy_comb'],'v_comb':simData['v_comb'],
                'wA':simData['wA'],'wB':simData['wB'],'wAVG':simData['wAVG'],
                'Re':simData['Re'],'time':simData['time']}
    df_sim = pd.DataFrame(data=dict_sim)
    HBData = pd.concat([HBData,df_sim],ignore_index=True)
    count += 1
    #print('Re ='+Re)
HBData = HBData.sort_values(by=['Re','time'])
HBData.to_csv(cwd_PYTHON+'/HBData.csv',index=False,sep=' ')
print(len(HBData['time']))
print('count = ',count)
print('Storing HBData is complete! Onto Analysis')

print('Storing All Data is complete! Onto Analysis')
                

Re:0.5
Re:0.6


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Re:0.7
Re:0.8
Re:0.9
Re:1.0
Re:2.0
Re:3.0
Re:4.0
Re:5.0
Re:5.5
Re:6.0
Re:6.5
Re:7.0
Re:7.5
Re:10.0
Re:12.5
Re:15.0
Re:17.5
Re:20.0
Re:25.0
Re:30.0
Re:35.0
Re:40.0
Re:50.0
Re:60.0
5096
count =  26
Storing VData is complete! Onto Analysis
Re:0.5
Re:1.0


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:60: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Re:2.0
Re:3.0
Re:4.0
Re:5.0
Re:5.5
Re:6.0
Re:6.5
Re:7.0
Re:7.5
Re:10.0
Re:12.5
Re:15.0
Re:17.5
Re:20.0
Re:25.0
Re:30.0
Re:35.0
Re:40.0
Re:50.0
Re:60.0
4283
count =  22
Storing OData is complete! Onto Analysis
Re:0.5


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:90: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Re:0.6
Re:0.7
Re:0.8
Re:0.9
Re:1.0
Re:2.0
Re:3.0
Re:4.0
Re:5.0
Re:5.5
Re:6.0
Re:6.5
Re:7.0
Re:7.5
Re:10.0
Re:12.5
Re:15.0
3618
count =  18
Storing SFData is complete! Onto Analysis
Re:0.5
Re:1.0


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:120: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Re:2.0
Re:3.0
Re:4.0
Re:5.0
Re:7.5
Re:10.0
Re:12.5
Re:15.0
Re:17.5
Re:20.0
Re:25.0
Re:30.0
Re:35.0
Re:40.0
Re:50.0
Re:60.0
3607
count =  18
Storing HBData is complete! Onto Analysis
Storing All Data is complete! Onto Analysis


#### Plot the following quantities:
##### (Theta_bw vs time), (Hmag vs time), (v_comb vs time)
#### After all are found, then get plateaus (long-time behavior) and plot vs Re
#### Get mean and std of each parameter being tracked
##### (Theta_bw vs Re), (Hmag vs. Re), (v_comb vs Re)

In [22]:
#Plotting Functions

def LabelSubPlots(ax, Re, config):

    csfont = {'fontname':'Times New Roman'}
    ax[0].set_title(r'$\theta_{bw}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[0].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[0].set_ylabel(r'$\theta_{bw}$',fontsize=12,**csfont)
    ax[1].set_title(r'$H_{bw}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[1].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[1].set_ylabel(r'$H_{bw}$',fontsize=12,**csfont)
    ax[2].set_title(r'v$_{comb}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[2].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[2].set_ylabel(r'v$_{comb}/Rf$',fontsize=12,**csfont)
    ax[3].set_title(r'$\omega_{avg}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[3].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[3].set_ylabel(r'$\omega_{avg}$',fontsize=12,**csfont)
    ax[0].set_xlim(0.0,20.0)
    if(config == 'SF'):
        ax[0].set_ylim(-10.0,10.0)
    elif(config == 'HB'):
        ax[0].set_ylim(160.0,200.0)
    ax[1].set_xlim(0.0,20.0)
    ax[1].set_ylim(2.5,6.0)
    ax[2].set_xlim(0.0,20.0)
    ax[2].set_ylim(0.0,0.1)
    ax[3].set_xlim(0.0,20.0)
    #ax[3].set_ylim(0.5,1.1)
    #ax[0].axis([-1.0,16.0,-19.0,16.0])
    #ax[1].axis([-1.0,16.0,-19.0,16.0])
    #ax[2].axis([-1.0,16.0,-19.0,16.0])
    
    return ax

def PlotvsTime(ax,data,varName,Re):
    ax.scatter(data['time'],data[varName],c='k',s=9)
    ax.plot(data['time'],data[varName],c='k')
    return ax
    

#### Plot Theta_BW, H_BW, and V_comb for 
#### Single File (SF) and Headbutt (HB)

In [23]:
%matplotlib inline

#Plot 1) Theta_bw vs time for each Re
#     2) Dist_bw vs time for each Re
#     3) vel_combined vs time for each Re (Might need to extract new info for this)
#fig, ax = plt.subplots(nrows=1,ncols=3,num=1,figsize=(19,6),dpi=250)


#Single File (SF)
SFData = pd.read_csv(cwd_PYTHON+'/SFData.csv',delimiter=' ')

for Re in SFReList:
    fig, ax = plt.subplots(nrows=1,ncols=4,num=1,figsize=(25,6),dpi=250)
    ReData = SFData[SFData['Re'] == float(Re)].copy()
    ReData = ReData.reset_index(drop=True)
    for idx in range(len(ReData['time'])):
        if(ReData.loc[idx,'Theta'] > 180.0):
            ReData.loc[idx,'Theta'] = ReData.loc[idx,'Theta'] - 360.0
    #print(ReData.head())
    ax = LabelSubPlots(ax,Re,'SF')
    ax[0] = PlotvsTime(ax[0],ReData,'Theta',Re)
    ax[1] = PlotvsTime(ax[1],ReData,'Hmag',Re)
    ax[2] = PlotvsTime(ax[2],ReData,'v_comb',Re)
    ax[3] = PlotvsTime(ax[3],ReData,'wAVG',Re)
    fig.tight_layout()
    #plt.show()
    #sys.exit(0)
    pathlib.Path(cwd_PYTHON+'/../Figures/SF/vsTime/').mkdir(parents=True, exist_ok=True)
    fig.savefig(cwd_PYTHON+'/../Figures/SF/vsTime/MonitoredParameters_Re'+Re+'.png')
    fig.clf()
    print('Re = %s done'%Re)
print('Plotting Monitored Parameters for SF is comnplete!')

#Headbutt (HB)
HBData = pd.read_csv(cwd_PYTHON+'/HBData.csv',delimiter=' ')

for Re in HBReList:
    fig, ax = plt.subplots(nrows=1,ncols=4,num=2,figsize=(25,6),dpi=250)
    ReData = HBData[HBData['Re'] == float(Re)].copy()
    ReData = ReData.reset_index(drop=True)
    #print(ReData.head())
    ax = LabelSubPlots(ax,Re,'HB')
    ax[0] = PlotvsTime(ax[0],ReData,'Theta',Re)
    ax[1] = PlotvsTime(ax[1],ReData,'Hmag',Re)
    ax[2] = PlotvsTime(ax[2],ReData,'v_comb',Re)
    ax[3] = PlotvsTime(ax[3],ReData,'wAVG',Re)
    fig.tight_layout()
    #plt.show()
    #sys.exit(0)
    pathlib.Path(cwd_PYTHON+'/../Figures/HB/vsTime/').mkdir(parents=True, exist_ok=True)
    fig.savefig(cwd_PYTHON+'/../Figures/HB/vsTime/MonitoredParameters_Re'+Re+'.png')
    fig.clf()
    print('Re = %s done'%Re)
print('Plotting Monitored Parameters for SF is comnplete!')
plt.close()


Re = 0.5 done
Re = 0.6 done
Re = 0.7 done
Re = 0.8 done
Re = 0.9 done
Re = 1.0 done
Re = 2.0 done
Re = 3.0 done
Re = 4.0 done
Re = 5.0 done
Re = 5.5 done
Re = 6.0 done
Re = 6.5 done
Re = 7.0 done
Re = 7.5 done
Re = 10.0 done
Re = 12.5 done
Re = 15.0 done
Plotting Monitored Parameters for SF is comnplete!
Re = 0.5 done
Re = 1.0 done
Re = 2.0 done
Re = 3.0 done
Re = 4.0 done
Re = 5.0 done
Re = 7.5 done
Re = 10.0 done
Re = 12.5 done
Re = 15.0 done
Re = 17.5 done
Re = 20.0 done
Re = 25.0 done
Re = 30.0 done
Re = 35.0 done
Re = 40.0 done
Re = 50.0 done
Re = 60.0 done
Plotting Monitored Parameters for SF is comnplete!


<Figure size 6250x1500 with 0 Axes>

In [7]:
%matplotlib inline

VData = pd.read_csv(cwd_PYTHON+'/VData.csv',delimiter=' ')
OData = pd.read_csv(cwd_PYTHON+'/OData.csv',delimiter=' ')
HBData = pd.read_csv(cwd_PYTHON+'/HBData.csv',delimiter=' ')
SFData = pd.read_csv(cwd_PYTHON+'/SFData.csv',delimiter=' ')

allData = [VData,OData,HBData,SFData]

for idx in range(len(allData)):
    
    

#Plot 1) Theta_bw vs time for each Re
#     2) Dist_bw vs time for each Re
#     3) vel_combined vs time for each Re (Might need to extract new info for this)
#fig, ax = plt.subplots(nrows=1,ncols=3,num=1,figsize=(19,6),dpi=250)
for Re in ReList:
    fig, ax = plt.subplots(nrows=1,ncols=4,num=1,figsize=(25,6),dpi=250)
    ReData = allData_sweep[allData_sweep['Re'] == float(Re)].copy()
    if(float(Re) == 35.0):
        ReData = ReData[ReData['time'] <= 18.0].copy()
    ReData = ReData.reset_index(drop=True)
    for idx in range(len(ReData['time'])):
        if(ReData.loc[idx,'Theta'] <= 90.0):
            ReData.loc[idx,'Theta'] *= -1.0
        else:
            ReData.loc[idx,'Theta'] = 360.0 - ReData.loc[idx,'Theta']
    #print(ReData.head())
    ax = LabelSubPlots(ax,Re)
    ax[0] = PlotvsTime(ax[0],ReData,'Theta',Re)
    ax[1] = PlotvsTime(ax[1],ReData,'Hmag',Re)
    ax[2] = PlotvsTime(ax[2],ReData,'v_comb',Re)
    ax[3] = PlotvsTime(ax[3],ReData,'par_ratio',Re)
    fig.tight_layout()
    #plt.show()
    #sys.exit(0)
    pathlib.Path(cwd_PYTHON+'/../Figures/V/vsTime/').mkdir(parents=True, exist_ok=True)
    fig.savefig(cwd_PYTHON+'/../Figures/V/vsTime/MonitoredParameters_Re'+Re+'.png')
    fig.clf()
    print('Re = %s done'%Re)
plt.close()


Re = 0.5 done
Re = 0.6 done
Re = 0.7 done
Re = 0.8 done
Re = 0.9 done
Re = 1.0 done
Re = 2.0 done
Re = 3.0 done
Re = 4.0 done
Re = 5.0 done
Re = 5.5 done
Re = 6.0 done
Re = 6.5 done
Re = 7.0 done
Re = 7.5 done
Re = 10.0 done
Re = 12.5 done
Re = 15.0 done
Re = 17.5 done
Re = 20.0 done
Re = 25.0 done
Re = 30.0 done
Re = 35.0 done
Re = 40.0 done
Re = 50.0 done
Re = 60.0 done


In [25]:
%matplotlib inline
from scipy import stats
csfont = {'fontname':'Times New Roman'}

def GetStats(data, delta, config):
    #Get mean and std of 
    # Theta_BW, H_BW, and Vcomb
    #Remove any outliers (H_BW jump)
    data['zscore'] = np.abs(stats.zscore(data['Hmag']))
    data = data[data['zscore'] < 3]
    data = data.reset_index(drop=True)
    #Means and stds
    theta_avg = data['Theta'].mean()
    theta_std = data['Theta'].std()
    #HBW_avg   = data['Hmag'].mean()*(RADIUSLARGE/delta)
    #HBW_std   = data['Hmag'].std()*(RADIUSLARGE/delta)
    HBW_avg   = data['Hmag'].mean()
    HBW_std   = data['Hmag'].std()
    #Hx and Hy
    Hx_avg   = data['Hx'].mean()
    Hx_std   = data['Hx'].std()
    Hy_avg   = data['Hy'].mean()
    Hy_std   = data['Hy'].std()
    if(config == 'O'):
        omega_avg = data['wAVG'].mean()
        omega_std = data['wAVG'].std()
    else:
        Vcomb_avg = data['v_comb'].mean()
        Vcomb_std = data['v_comb'].std()
    if(config == 'O'):
        return (theta_avg, theta_std, HBW_avg, HBW_std, Hx_avg, Hx_std, Hy_avg, Hy_std, omega_avg, omega_std)
    else:
        return (theta_avg, theta_std, HBW_avg, HBW_std, Hx_avg, Hx_std, Hy_avg, Hy_std, Vcomb_avg, Vcomb_std)

def LabelSubplots_Re(ax,config):
    csfont = {'fontname':'Times New Roman'}
    ax[0].set_title(r'%s: $\theta_{bw}$ vs. Re: 2$\sigma$'%config,fontsize=15,**csfont)
    ax[0].set_xlabel(r'Re',fontsize=12,**csfont)
    ax[0].set_ylabel(r'$\theta_{bw}$',fontsize=12,**csfont)
    ax[1].set_title(r'$H_{bw}$ vs. Re: 2$\sigma$',fontsize=15,**csfont)
    ax[1].set_xlabel(r'Re',fontsize=12,**csfont)
    ax[1].set_ylabel(r'$H_{bw}$ ($\delta$)',fontsize=12,**csfont)
    #ax[1].set_ylabel(r'$H_{bw}$ (R)',fontsize=12,**csfont)
    ax[2].set_xlabel(r'Re',fontsize=12,**csfont)
    if(config == 'O'):
        ax[2].set_title(r'$\omega_{avg}$ vs. Re: 2$\sigma$',fontsize=15,**csfont)
        ax[2].set_ylabel(r'$\omega_{avg}$ (deg/s)',fontsize=12,**csfont)
    else:
        ax[2].set_title(r'v$_{comb}$ vs. Re: 2$\sigma$',fontsize=15,**csfont)
        ax[2].set_ylabel(r'v$_{comb}/Rf$',fontsize=12,**csfont)
    return ax

def PlotMonitoredParameters(ax,Re,thetaBW,HBW,Vcomb,color):
    ax[0].errorbar(Re, thetaBW[:len(Re),0], yerr=2.0*thetaBW[:len(Re),1],c=color)
    ax[0].scatter(Re, thetaBW[:len(Re),0],c=color,s=12)
    ax[1].errorbar(Re, HBW[:len(Re),0], yerr=2.0*HBW[:len(Re),1],c=color)
    ax[1].scatter(Re, HBW[:len(Re),0],c=color,s=12)
    ax[2].errorbar(Re, Vcomb[:len(Re),0], yerr=2.0*Vcomb[:len(Re),1],c=color)
    ax[2].scatter(Re, Vcomb[:len(Re),0],c=color,s=12)
    return ax

#V-Shape (V)
VData = pd.read_csv(cwd_PYTHON+'/VData.csv',delimiter=' ')

#Construct time_ES list
time_ES = [2.5,10.0,10.0,8.5,8.0,7.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.5,3.0,3.0,2.5,2.5,2.5,2.5,2.0,2.0,2.0,2.0,2.0]
#Given the time_ES (time the end state begins), calculate the mean and std dev of the following parameters
#theta_BW, H_BW, v_comb for each Re
#Save them in np arrays
#Plot parameters vs Re
    
#Arrays (0 = mean, 1 = std dev)
thetaBW_V = np.zeros((26,2))
HBW_V     = np.zeros((26,2))
Hx_V      = np.zeros((26,2))
Hy_V      = np.zeros((26,2))
Vcomb_V   = np.zeros((26,2))
theta2_V  = np.zeros((2,2))
HBW2_V    = np.zeros((2,2))
Hx2_V     = np.zeros((2,2))
Hy2_V     = np.zeros((2,2))
Vcomb2_V  = np.zeros((2,2))

f = open('PairStats_V.csv','w')
f.write('Re theta_avg theta_std Hx_avg Hx_std Hy_avg Hy_std Hmag_avg Hmag_std\n')

for idx in range(len(VReList)):
    Re = float(VReList[idx])
    ReData = VData[VData['Re'] == Re].copy()
    ReData = ReData.reset_index(drop=True)
    for jdx in range(len(ReData['time'])):
        if(ReData.loc[jdx,'Theta'] <= 90.0):
            ReData.loc[jdx,'Theta'] *= -1.0
        else:
            ReData.loc[jdx,'Theta'] = 360.0 - ReData.loc[jdx,'Theta']
    #Calculate Delta for each Re
    delta = np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/Re)
    if(Re <= 35.0):
        ReData = ReData[ReData['time'] >= 10.0].copy()
        if(Re == 35.0):
            ReData = ReData[ReData['time'] <= 18.0].copy()
        ReData = ReData.reset_index(drop=True)
        #Means and stds
        thetaBW_V[idx,0], thetaBW_V[idx,1], HBW_V[idx,0], HBW_V[idx,1], Hx_V[idx,0], Hx_V[idx,1], Hy_V[idx,0], Hy_V[idx,1], Vcomb_V[idx,0], Vcomb_V[idx,1] = GetStats(ReData, delta,'V')
        
    elif(Re == 40.0):
        ReData1 = ReData[ReData['time'] >= 8.0].copy()
        ReData1 = ReData1[ReData1['time'] <= 9.0].copy()
        ReData2 = ReData[ReData['time'] >= 12.5].copy()
        ReData2 = ReData2[ReData2['time'] <= 13.5].copy()
        ReData1 = ReData1.reset_index(drop=True)
        ReData2 = ReData2.reset_index(drop=True)
        #Means and stds
        thetaBW_V[idx,0], thetaBW_V[idx,1], HBW_V[idx,0], HBW_V[idx,1], Hx_V[idx,0], Hx_V[idx,1], Hy_V[idx,0], Hy_V[idx,1], Vcomb_V[idx,0], Vcomb_V[idx,1] = GetStats(ReData1, delta,'V')
        theta2_V[0,0], theta2_V[0,1], HBW2_V[0,0], HBW2_V[0,1], Hx2_V[0,0], Hx2_V[0,1], Hy2_V[0,0], Hy2_V[0,1], Vcomb2_V[0,0], Vcomb2_V[0,1] = GetStats(ReData2, delta,'V')
        print(theta2_V[0,0])
        
    elif(Re == 50.0):
        ReData1 = ReData[ReData['time'] >= 12.5].copy()
        ReData1 = ReData1[ReData1['time'] <= 17.5].copy()
        ReData2 = ReData[ReData['time'] >= 4.5].copy()
        ReData2 = ReData2[ReData2['time'] <= 8.5].copy()
        ReData1 = ReData1.reset_index(drop=True)
        ReData2 = ReData2.reset_index(drop=True)
        #Means and stds
        thetaBW_V[idx,0], thetaBW_V[idx,1], HBW_V[idx,0], HBW_V[idx,1], Hx_V[idx,0], Hx_V[idx,1], Hy_V[idx,0], Hy_V[idx,1], Vcomb_V[idx,0], Vcomb_V[idx,1] = GetStats(ReData1, delta,'V')
        theta2_V[1,0], theta2_V[1,1], HBW2_V[1,0], HBW2_V[1,1], Hx2_V[1,0], Hx2_V[1,1], Hy2_V[1,0], Hy2_V[1,1], Vcomb2_V[1,0], Vcomb2_V[1,1] = GetStats(ReData2, delta,'V')

    else:
        print('Re = 60 has no stable end state: Collision') 
        
    if(float(VReList[idx]) < 35.0):
        Vcomb_V[idx,0] *= -1.0
        
    #Record Re, theta, Hx, Hy, Hmag into csv for Hong
    f.write('%.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e\n'%
            (Re,thetaBW_V[idx,0],thetaBW_V[idx,1],Hx_V[idx,0],Hx_V[idx,1],Hy_V[idx,0],Hy_V[idx,1],HBW_V[idx,0],HBW_V[idx,1]))
f.close()
    
#Plot V-Shape statistics
fig, ax = plt.subplots(nrows=1,ncols=3,num=1,figsize=(19,6),dpi=250)
ax = LabelSubplots_Re(ax,'V')

#Plot Parameters with error bars
nRe = 25
VRe = [float(VReList[a]) for a in range(nRe)]
#floatRe = [float(a) for a in VReList]
#nRe = len(floatRe) - 1
ax = PlotMonitoredParameters(ax,VRe,thetaBW_V,HBW_V,Vcomb_V,'black')
#Add meta-stable values
ax[0].errorbar([40.0], theta2_V[0,0], yerr=2.0*theta2_V[0,1],c='tab:blue')
ax[0].scatter([40.0], theta2_V[0,0],c='tab:blue',s=12)
ax[1].errorbar([40.0], HBW2_V[0,0], yerr=2.0*HBW2_V[0,1],c='tab:blue')
ax[1].scatter([40.0], HBW2_V[0,0],c='tab:blue',s=12)
ax[2].errorbar([40.0], Vcomb2_V[0,0], yerr=2.0*Vcomb2_V[0,1],c='tab:blue')
ax[2].scatter([40.0], Vcomb2_V[0,0],c='tab:blue',s=12)
ax[0].errorbar([50.0], theta2_V[1,0], yerr=2.0*theta2_V[1,1],c='tab:blue')
ax[0].scatter([50.0], theta2_V[1,0],c='tab:blue',s=12)
ax[1].errorbar([50.0], HBW2_V[1,0], yerr=2.0*HBW2_V[1,1],c='tab:blue')
ax[1].scatter([50.0], HBW2_V[1,0],c='tab:blue',s=12)
ax[2].errorbar([50.0], Vcomb2_V[1,0], yerr=2.0*Vcomb2_V[1,1],c='tab:blue')
ax[2].scatter([50.0], Vcomb2_V[1,0],c='tab:blue',s=12)
ax[0].axis([0.0,55.0,0.0,270.0])
ax[1].axis([0.0,55.0,5.0,30.0])
#ax[1].axis([0.0,55.0,2.0,6.0])
ax[2].axis([0.0,55.0,-0.25,0.375])
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/V/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/V/V_MonitoredStatistics.png')
fig.clf()
print(thetaBW_V[:,0])
print(HBW_V[:,0])
print('Plotting V vs. Re is complete!')

#Orbit (O)
OData = pd.read_csv(cwd_PYTHON+'/OData.csv',delimiter=' ')
#Construct time_ES list
ORe = [6.0,6.5,7.0,7.5,10.0,12.5,15.0,17.5,20.0,25.0,30.0]
time_ES = [12.0,15.0,6.0,5.0,4.0,4.0,3.5,3.5,3.0,3.0,3.0]
nRe = 11

#Arrays (0 = mean, 1 = std dev)
thetaBW_O = np.zeros((nRe,2))
HBW_O     = np.zeros((nRe,2))
Hx_O      = np.zeros((nRe,2))
Hy_O      = np.zeros((nRe,2))
omega_O   = np.zeros((nRe,2))

f = open('PairStats_O.csv','w')
f.write('Re theta_avg theta_std Hx_avg Hx_std Hy_avg Hy_std Hmag_avg Hmag_std\n')

for idx in range(nRe):
    Re = float(ORe[idx])
    ReData = OData[OData['Re'] == Re].copy()
    ReData = ReData[ReData['time'] >= time_ES[idx]].copy()
    ReData = ReData.reset_index(drop=True)
    ReData['Theta'] = 360.0 - ReData['Theta']
    #Calculate Delta for each Re
    delta = np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/Re)
    thetaBW_O[idx,0], thetaBW_O[idx,1], HBW_O[idx,0], HBW_O[idx,1], Hx_O[idx,0], Hx_O[idx,1], Hy_O[idx,0], Hy_O[idx,1],omega_O[idx,0], omega_O[idx,1] = GetStats(ReData, delta,'O')
    
    #Record Re, theta, Hx, Hy, Hmag into csv for Hong
    f.write('%.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e\n'%
            (Re,thetaBW_O[idx,0],thetaBW_O[idx,1],Hx_O[idx,0],Hx_O[idx,1],Hy_O[idx,0],Hy_O[idx,1],HBW_O[idx,0],HBW_O[idx,1]))
f.close()
    
#Plot parameters vs. Re (include std as error bar)
fig, ax = plt.subplots(nrows=1,ncols=3,num=2,figsize=(19,6),dpi=250)
ax = LabelSubplots_Re(ax,'O')

#Plot Parameters with error bars
ax = PlotMonitoredParameters(ax,ORe,thetaBW_O,HBW_O,omega_O,'black')

ax[0].axis([5.0,35.0,170.0,190.0])
ax[1].axis([5.0,35.0,5.0,30.0])
#ax[1].axis([5.0,35.0,2.0,6.0])
#ax[2].axis([0.0,60.0,-0.5,0.5])
ax[2].set_xlim(5.0,35.0)
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/O/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/O/O_MonitoredStatistics.png')
fig.clf()
print(thetaBW_O[:,0])
print(HBW_O[:,0])
print('Plotting O vs. Re is complete!')

#Single File (SF)
SFData = pd.read_csv(cwd_PYTHON+'/SFData.csv',delimiter=' ')
nRe = 15
time_ES = [2.5 for a in range(nRe)]

thetaBW_SF = np.zeros((nRe,2))
HBW_SF     = np.zeros((nRe,2))
Hx_SF      = np.zeros((nRe,2))
Hy_SF      = np.zeros((nRe,2))
Vcomb_SF   = np.zeros((nRe,2))

f = open('PairStats_SF.csv','w')
f.write('Re theta_avg theta_std Hx_avg Hx_std Hy_avg Hy_std Hmag_avg Hmag_std\n')

for idx in range(nRe):
    Re = float(SFReList[idx])
    ReData = SFData[SFData['Re'] == Re].copy()
    ReData = ReData.reset_index(drop=True)
    #Calculate Delta for each Re
    delta = np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/Re)
    ReData = ReData[ReData['time'] >= time_ES[idx]]
    ReData = ReData.reset_index(drop=True)
    for jdx in range(len(ReData['time'])):
        if(ReData.loc[jdx,'Theta'] > 180.0):
            ReData.loc[jdx,'Theta'] = ReData.loc[jdx,'Theta'] - 360.0
    #Means and stds
    thetaBW_SF[idx,0], thetaBW_SF[idx,1], HBW_SF[idx,0], HBW_SF[idx,1], Hx_SF[idx,0], Hx_SF[idx,1], Hy_SF[idx,0], Hy_SF[idx,1], Vcomb_SF[idx,0], Vcomb_SF[idx,1] = GetStats(ReData, delta,'SF')
    
    #Record Re, theta, Hx, Hy, Hmag into csv for Hong
    f.write('%.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e\n'%
            (Re,thetaBW_SF[idx,0],thetaBW_SF[idx,1],Hx_SF[idx,0],Hx_SF[idx,1],Hy_SF[idx,0],Hy_SF[idx,1],HBW_SF[idx,0],HBW_SF[idx,1]))
f.close()
    
Vcomb_SF[:,0] = -1.0*Vcomb_SF[:,0]

    
fig, ax = plt.subplots(nrows=1,ncols=3,num=3,figsize=(19,6),dpi=250)
ax = LabelSubplots_Re(ax,'SF')

#Plot Parameters with error bars
SFRe = [float(SFReList[a]) for a in range(nRe)]
ax = PlotMonitoredParameters(ax,SFRe,thetaBW_SF,HBW_SF,Vcomb_SF,'black')
ax[0].axis([0.0,8.0,-5.0,5.0])
ax[1].axis([0.0,8.0,5.0,30.0])
#ax[1].axis([0.0,8.0,2.0,6.0])
ax[2].axis([0.0,8.0,-0.1,0.05])
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/SF/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/SF/SF_MonitoredStatistics.png')
fig.clf()
print(thetaBW_SF[:,0])
print(HBW_SF[:,0])
print('Plotting SF vs. Re is complete!')

    
#Headbutt (HB)
HBData = pd.read_csv(cwd_PYTHON+'/HBData.csv',delimiter=' ')
nRe = 11
time_ES = [2.5 for a in range(nRe)]

thetaBW_HB = np.zeros((nRe,2))
HBW_HB     = np.zeros((nRe,2))
Hx_HB      = np.zeros((nRe,2))
Hy_HB      = np.zeros((nRe,2))
Vcomb_HB   = np.zeros((nRe,2))

f = open('PairStats_HB.csv','w')
f.write('Re theta_avg theta_std Hx_avg Hx_std Hy_avg Hy_std Hmag_avg Hmag_std\n')

for idx in range(1,nRe+1):
    Re = float(HBReList[idx])
    ReData = HBData[HBData['Re'] == Re].copy()
    ReData = ReData.reset_index(drop=True)
    #Calculate Delta for each Re
    delta = np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/Re)
    ReData = ReData[ReData['time'] >= time_ES[idx-1]]
    ReData = ReData.reset_index(drop=True)
    #Means and stds
    thetaBW_HB[idx-1,0], thetaBW_HB[idx-1,1], HBW_HB[idx-1,0], HBW_HB[idx-1,1], Hx_HB[idx-1,0],Hx_HB[idx-1,1],Hy_HB[idx-1,0],Hy_HB[idx-1,1], Vcomb_HB[idx-1,0], Vcomb_HB[idx-1,1] = GetStats(ReData, delta,'HB')

    #Record Re, theta, Hx, Hy, Hmag into csv for Hong
    f.write('%.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e\n'%
            (Re,thetaBW_HB[idx-1,0],thetaBW_HB[idx-1,1],Hx_HB[idx-1,0],Hx_HB[idx-1,1],Hy_HB[idx-1,0],Hy_HB[idx-1,1],HBW_HB[idx-1,0],HBW_HB[idx-1,1]))
f.close()
  
fig, ax = plt.subplots(nrows=1,ncols=3,num=4,figsize=(19,6),dpi=250)
ax = LabelSubplots_Re(ax,'HB')

#Plot Parameters with error bars
HBRe = [float(HBReList[a]) for a in range(1,nRe+1)]
ax = PlotMonitoredParameters(ax,HBRe,thetaBW_HB,HBW_HB,Vcomb_HB,'black')
ax[0].axis([0.0,25.0,175.0,185.0])
ax[1].axis([0.0,25.0,5.0,30.0])
#ax[1].axis([0.0,25.0,2.0,6.0])
ax[2].axis([0.0,25.0,-0.05,0.1])
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/HB/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/HB/HB_MonitoredStatistics.png')
fig.clf()
print(thetaBW_HB[:,0])
print(HBW_HB[:,0])
print('Plotting HB vs. Re is complete!')

#Plot Everything on the same figure!
fig, ax = plt.subplots(nrows=1,ncols=1,num=5,figsize=(8,6),dpi=250)
#ax.set_title(r'$H_{bw}$ vs. Re',fontsize=18,**csfont)
ax.set_xlabel(r'Re',fontsize=16,**csfont)
#ax.set_ylabel(r'$H_{bw}$ ($\delta$)',fontsize=16,**csfont)
ax.set_ylabel(r'$H_{bw}$ (R)',fontsize=16,**csfont)
#Plot H_bw vs Re for each config
colorV = (117.0/255.0,112.0/255.0,179.0/255.0)
colorO = (231.0/255.0,41.0/255.0,138.0/255.0)
colorSF = (27.0/255.0,158.0/255.0,119.0/255.0)
colorHB = (217.0/255.0,95.0/255.0,2.0/255.0)
ax.errorbar(VRe, HBW_V[:len(VRe),0], yerr=2.0*HBW_V[:len(VRe),1],c=colorV)
ax.scatter(VRe, HBW_V[:len(VRe),0],c=colorV,s=12)
ax.errorbar(ORe, HBW_O[:len(ORe),0], yerr=2.0*HBW_O[:len(ORe),1],c=colorO)
ax.scatter(ORe, HBW_O[:len(ORe),0],c=colorO,s=12)
ax.errorbar(SFRe, HBW_SF[:len(SFRe),0], yerr=2.0*HBW_SF[:len(SFRe),1],c=colorSF)
ax.scatter(SFRe, HBW_SF[:len(SFRe),0],c=colorSF,s=12)
ax.errorbar(HBRe, HBW_HB[:len(HBRe),0], yerr=2.0*HBW_HB[:len(HBRe),1],c=colorHB)
ax.scatter(HBRe, HBW_HB[:len(HBRe),0],c=colorHB,s=12)
#ax.axis([0.0,55.0,5.0,30.0])
ax.axis([0.0,55.0,2.0,7.0])
#Axes Parameters
ax.tick_params(which='major',axis='both',direction='in',length=10,width=1)
ax.tick_params(which='minor',axis='both',direction='in',length=6,width=0.75)
#ax.set_axisbelow(False)
#ax.set_xticks(np.arange(0.0,1.1,step=0.2))
#ax.xaxis.set_minor_locator(MultipleLocator(0.1))
#ax.yaxis.set_minor_locator(MultipleLocator(0.01))
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/Distances/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/Distances/Hbw_R.png')
fig.clf()
print('Plotting H_bw vs Re for all configs complete!')

#Plot Everything on the same figure!
fig, ax = plt.subplots(nrows=1,ncols=1,num=5,figsize=(8,6),dpi=250)
#ax.set_title(r'$H_{bw}$ vs. Re',fontsize=18,**csfont)
ax.set_xlabel(r'Re',fontsize=16,**csfont)
ax.set_ylabel(r'$\theta_{bw}$',fontsize=16,**csfont)
#Plot H_bw vs Re for each config
colorV = (117.0/255.0,112.0/255.0,179.0/255.0)
colorO = (231.0/255.0,41.0/255.0,138.0/255.0)
colorSF = (27.0/255.0,158.0/255.0,119.0/255.0)
colorHB = (217.0/255.0,95.0/255.0,2.0/255.0)
ax.errorbar(VRe, thetaBW_V[:len(VRe),0], yerr=2.0*thetaBW_V[:len(VRe),1],c=colorV)
ax.scatter(VRe, thetaBW_V[:len(VRe),0],c=colorV,s=12)
ax.errorbar(ORe, thetaBW_O[:len(ORe),0], yerr=2.0*thetaBW_O[:len(ORe),1],c=colorO)
ax.scatter(ORe, thetaBW_O[:len(ORe),0],c=colorO,s=12)
ax.errorbar(SFRe, thetaBW_SF[:len(SFRe),0], yerr=2.0*thetaBW_SF[:len(SFRe),1],c=colorSF)
ax.scatter(SFRe, thetaBW_SF[:len(SFRe),0],c=colorSF,s=12)
ax.errorbar(HBRe, thetaBW_HB[:len(HBRe),0], yerr=2.0*thetaBW_HB[:len(HBRe),1],c=colorHB)
ax.scatter(HBRe, thetaBW_HB[:len(HBRe),0],c=colorHB,s=12)
ax.plot([0.0,60.0],[0.0,0.0],c='k')
#ax.axis([0.0,55.0,5.0,30.0])
ax.axis([0.0,55.0,-5.0,185.0])
#Axes Parameters
ax.tick_params(which='major',axis='both',direction='in',length=6,width=1)
ax.tick_params(which='minor',axis='both',direction='in',length=4,width=0.75)
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/Angles/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/Angles/thetabw_all.png')
fig.clf()
print('Plotting Theta_bw vs Re for all configs complete!')

#Plot H_bw vs delta for all configs
#Include linear regressions
#Make log-log
Vdelta = np.array([np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/float(a)) for a in VRe])
Odelta = np.array([np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/float(a)) for a in ORe])
SFdelta = np.array([np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/float(a)) for a in SFRe])
HBdelta = np.array([np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/float(a)) for a in HBRe])

#Find Relationship between H_bw and \delta
#Use HBW_stats and deltaValues
#Find Linear Regression
slopeV, interceptV, r_value, p_value, std_err = stats.linregress(np.log10(Vdelta),np.log10(HBW_V[:len(Vdelta),0]))
yFitV = 10.0**(interceptV)*Vdelta**slopeV
print("="*40)
print(r'Power Law: $\delta$ exponent')
print('N = ',len(Vdelta))
print('exponent = %.3f'%slopeV)
print('intercept = %.3f'%interceptV)
print('r_value = %.3f'%r_value)
print("-"*40)
slopeO, interceptO, r_value, p_value, std_err = stats.linregress(np.log10(Odelta),np.log10(HBW_O[:,0]))
yFitO = 10.0**(interceptO)*Odelta**slopeO
print("="*40)
print(r'Power Law: $\delta$ exponent')
print('N = ',len(Odelta))
print('exponent = %.3f'%slopeO)
print('intercept = %.3f'%interceptO)
print('r_value = %.3f'%r_value)
print("-"*40)
slopeSF, interceptSF, r_value, p_value, std_err = stats.linregress(np.log10(SFdelta),np.log10(HBW_SF[:,0]))
yFitSF = 10.0**(interceptSF)*SFdelta**slopeSF
print("="*40)
print(r'Power Law: $\delta$ exponent')
print('N = ',len(SFdelta))
print('exponent = %.3f'%slopeSF)
print('intercept = %.3f'%interceptSF)
print('r_value = %.3f'%r_value)
print("-"*40)
slopeHB, interceptHB, r_value, p_value, std_err = stats.linregress(np.log10(HBdelta),np.log10(HBW_HB[:,0]))
yFitHB = 10.0**(interceptHB)*HBdelta**slopeHB
print("="*40)
print(r'Power Law: $\delta$ exponent')
print('N = ',len(HBdelta))
print('exponent = %.3f'%slopeV)
print('intercept = %.3f'%interceptV)
print('r_value = %.3f'%r_value)
print("-"*40)

#Plot Everything on the same figure! LOG-LOG
fig, ax = plt.subplots(nrows=1,ncols=1,num=5,figsize=(8,6),dpi=250)
#ax.set_title(r'$H_{bw}$ vs. Re: $H_{bw}$ $\propto$ $\delta^x$',fontsize=18,**csfont)
ax.set_xlabel(r'Re',fontsize=16,**csfont)
#ax.set_ylabel(r'$H_{bw}$ ($\delta$)',fontsize=16,**csfont)
ax.set_ylabel(r'$H_{bw}$ (R)',fontsize=16,**csfont)
#Plot H_bw vs Re for each config
#logVRe, logORe, logSFRe, logHBRe = np.log10(VRe), np.log10(ORe), np.log10(SFRe), np.log10(HBRe)
#logHBW_V, logHBW_O, logHBW_SF, logHBW_HB = np.log10(HBW_V), np.log10(HBW_O), np.log10(HBW_SF), np.log10(HBW_HB)
ax.loglog(VRe, HBW_V[:len(VRe),0],c=colorV,label=None)
ax.scatter(VRe, HBW_V[:len(VRe),0],c=colorV,s=12,label=None)
ax.plot(VRe,yFitV,c=colorV,ls='--',label=r'$\delta^x$: x = %.2f'%slopeV)
ax.loglog(ORe, HBW_O[:len(ORe),0], c=colorO,label=None)
ax.scatter(ORe, HBW_O[:len(ORe),0],c=colorO,s=12,label=None)
ax.plot(ORe,yFitO,c=colorO,ls='--',label=r'$\delta^x$: x = %.2f'%slopeO)
ax.loglog(SFRe, HBW_SF[:len(SFRe),0], c=colorSF,label=None)
ax.scatter(SFRe, HBW_SF[:len(SFRe),0],c=colorSF,s=12,label=None)
ax.plot(SFRe,yFitSF,c=colorSF,ls='--',label=r'$\delta^x$: x = %.2f'%slopeSF)
ax.loglog(HBRe, HBW_HB[:len(HBRe),0], c=colorHB,label=None)
ax.scatter(HBRe, HBW_HB[:len(HBRe),0],c=colorHB,s=12,label=None)
ax.plot(HBRe,yFitHB,c=colorHB,ls='--',label=r'$\delta^x$: x = %.2f'%slopeHB)
#ax.axis([np.log10(0.2),np.log10(55.0),np.log10(5.0),np.log10(30.0)])
#ax.axis([0.25,55.0,5.0,30.0])
ax.axis([0.25,55.0,2.0,7.0])
ax.legend(loc='best',fontsize='large')
#Axes Parameters
ax.tick_params(which='major',axis='both',direction='in',length=10,width=1)
ax.tick_params(which='minor',axis='both',direction='in',length=6,width=0.75)
#ax.set_axisbelow(False)
#ax.set_xticks(np.arange(0.0,1.1,step=0.2))
#ax.xaxis.set_minor_locator(MultipleLocator(0.1))
#ax.yaxis.set_minor_locator(MultipleLocator(0.01))
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/Distances').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/Distances/Hbw_R_LL.png')
fig.clf()
print('Plotting H_bw vs Re for all configs LL complete!')

plt.close()


237.82625414274423
Re = 60 has no stable end state: Collision
[180.09892524  65.95075957  64.56026039  63.54002717  62.83949667
  61.8275493   53.88553476  46.50801982  40.04586625  35.40131712
  33.73197894  32.5395472   31.57672414  30.828941    30.18803429
  28.01488088  26.30636703  25.48477481  24.40688458  23.185694
  20.30017813  15.2172058   13.85120979  10.71209433  11.05847754
   0.        ]
[5.68638064 6.26613107 6.0484137  5.87301354 5.72661392 5.59762148
 4.76950138 4.26950245 3.89051386 3.60044619 3.48694721 3.39121677
 3.31128943 3.24313144 3.18493604 2.98166461 2.8472599  2.74702086
 2.67186571 2.61912629 2.54928434 2.48990009 2.34336203 2.34626346
 2.34706496 0.        ]
Plotting V vs. Re is complete!
[190.75237329 181.34228812 180.39921703 180.47806464 180.13103454
 180.16568199 180.12451069 179.95045817 180.50452501 182.22602664
 183.92371518]
[3.45689747 3.34008092 3.25634501 3.23923154 3.06836688 2.98512284
 2.93281939 2.89318969 2.86140185 2.8050702  2.77078324]
P

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

[179.97772219 179.99141112 180.02374066 179.92047278 179.98292982
 180.13630738 180.21328196 180.20566752 180.26941622 180.17724013
 180.18191808]
[3.68971472 3.44690615 3.38102511 3.34185848 3.32876086 3.31740482
 3.31664317 3.35644943 3.39292816 3.52182675 3.6147564 ]
Plotting HB vs. Re is complete!


'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

Plotting H_bw vs Re for all configs complete!


...

Plotting Theta_bw vs Re for all configs complete!
Power Law: $\delta$ exponent
N =  25
exponent = 0.470
intercept = 2.112
r_value = 0.991
----------------------------------------
Power Law: $\delta$ exponent
N =  11
exponent = 0.259
intercept = 1.390
r_value = 0.976
----------------------------------------
Power Law: $\delta$ exponent
N =  15
exponent = -0.102
intercept = 0.361
r_value = -0.863
----------------------------------------
Power Law: $\delta$ exponent
N =  11
exponent = 0.470
intercept = 2.112
r_value = 0.194
----------------------------------------
Plotting H_bw vs Re for all configs LL complete!


<Figure size 4750x1500 with 0 Axes>

<Figure size 4750x1500 with 0 Axes>

<Figure size 4750x1500 with 0 Axes>

<Figure size 4750x1500 with 0 Axes>

In [20]:
colorV = (117.0/255.0,112.0/255.0,179.0/255.0)
colorO = (231.0/255.0,41.0/255.0,138.0/255.0)
colorSF = (27.0/255.0,158.0/255.0,119.0/255.0)
colorHB = (217.0/255.0,95.0/255.0,2.0/255.0)
#Compare v_comb to single swimmer velocity
cwd_Single = cwd_PYTHON+'/../../SingleSwimmer/Python/'
singleData = pd.read_csv(cwd_Single+'SingleSwimmer_VelData.csv',delimiter=',')
singleData = singleData[singleData['Re'] <= 50.0].copy()
singleData['v_avg'] = singleData['v_avg']*RADIUSSMALL/RADIUSLARGE
singleData = singleData.reset_index(drop=True)
#Plot Both v_comb and v_single on same axes
fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(8,6),dpi=250)
#ax.set_title(r'$v_{avg}$ vs. Re: Pair vs. Single',fontsize=15,**csfont)
ax.set_xlabel(r'Re',fontsize=16,**csfont)
#ax.set_xlabel(r'$\delta$ (m)',fontsize=12,**csfont)
ax.set_ylabel(r'$\langle v \rangle$ (v/Rf)',fontsize=16,**csfont)
ax.plot([0.0,65.0],[0.0,0.0] ,color='k')
#V-Shape
ax.errorbar(VRe, Vcomb_V[:len(VRe),0], yerr=2.0*Vcomb_V[:len(VRe),1],c=colorV,label='In-Tandem V')
ax.scatter(VRe, Vcomb_V[:len(VRe),0],c=colorV,s=12,label=None)
#Add meta-stable values
#ax.errorbar([40.0], Vcomb2_V[0,0], yerr=2.0*Vcomb2_V[0,1],c='tab:blue',label=None)
#ax.scatter([40.0], Vcomb2_V[0,0],c='tab:blue',s=12,label=None)
#ax.errorbar([50.0], Vcomb2_V[1,0], yerr=2.0*Vcomb2_V[1,1],c='tab:blue',label=None)
#ax.scatter([50.0], Vcomb2_V[1,0],c='tab:blue',s=12,label=None)
#Single File (In-Line)
ax.errorbar(SFRe, Vcomb_SF[:len(SFRe),0], yerr=2.0*Vcomb_SF[:len(SFRe),1],c=colorSF,label='In-Line')
ax.scatter(SFRe, Vcomb_SF[:len(SFRe),0],c=colorSF,s=12,label=None)
#Single Bot
ax.errorbar(singleData['Re'],singleData['v_avg'],yerr=2.0*singleData['v_std'],c='k',label='Single Bot')
ax.scatter(singleData['Re'],singleData['v_avg'],c='k',s=12,label=None)
ax.axis([0.0,55.0,-0.2,0.375])
ax.legend(loc='best',fontsize='medium')
#Axes Parameters
ax.tick_params(which='major',axis='both',direction='in',length=10,width=1,zorder=10)
ax.tick_params(which='minor',axis='both',direction='in',length=6,width=0.75)
#ax.set_axisbelow(False)
#ax.set_xticks(np.arange(0.0,1.1,step=0.2))
#ax.xaxis.set_minor_locator(MultipleLocator(0.1))
#ax.yaxis.set_minor_locator(MultipleLocator(0.01))
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/Velocities/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/Velocities/CompareVel_SinglePair.png')
fig.clf()
plt.close()
print('Plotting Pair vs. Single Velocities is complete')



'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.


Plotting Pair vs. Single Velocities is complete


In [32]:
%matplotlib inline
colorV = (117.0/255.0,112.0/255.0,179.0/255.0)
#Plot Avg Vel and Theta_BW on the same plot
#Using two different axes
fig = plt.figure(num=1,figsize=(6,4),dpi=200)
ax = fig.add_subplot(111)
csfont = {'fontname':'Times New Roman'}
#ax.set_title(r'Center of Mass',fontsize=20)
ax.set_xlabel(r'Re',fontsize=14,**csfont)
ax.set_ylabel(r'$\langle v \rangle$ (v/Rf)',fontsize=14,**csfont)
#Horizontal Lines
ax.plot([0.0,65.0],[0.0,0.0] ,color='k',lw=2)
ax.errorbar(VRe,Vcomb_V[:len(VRe),0],yerr=2.0*Vcomb_V[:len(VRe),1],c=colorV,zorder=5)
ax.scatter(VRe,Vcomb_V[:len(VRe),0],color=colorV,s=12,zorder=5)
ax2 = ax.twinx()
ax2.set_ylabel(r'$\theta_{BW}$',fontsize=14,**csfont,color=colorV)
#nRe = len(floatRe)-1
ax2.errorbar(VRe,thetaBW_V[:len(VRe),0],yerr=2.0*thetaBW_V[:len(VRe),1],c='k',zorder=1)
ax2.scatter(VRe,thetaBW_V[:len(VRe),0],c='k',s=12,zorder=1)
ax2.tick_params(which='major',labelcolor='k')
ax2.tick_params(which='minor',labelcolor='k')
ax.tick_params(which='major',labelcolor=colorV)
ax.tick_params(which='minor',labelcolor=colorV)
ax.set_xlim(0.0,65.0)
ax.set_ylim(-0.5,0.75)
ax2.set_ylim(0.0,185.0)
#Axes Parameters
ax.tick_params(which='major',axis='both',direction='in',length=6,width=1,zorder=10)
ax.tick_params(which='minor',axis='both',direction='in',length=4,width=0.75)
ax2.tick_params(which='major',axis='both',direction='in',length=6,width=1,zorder=10)
ax2.tick_params(which='minor',axis='both',direction='in',length=4,width=0.75)
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/Angles/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/Angles/AvgVel_ThetaBW_V.png')
ax.set_xlim(0.0,20.0)
ax.set_ylim(-0.5,0.0)
ax2.set_ylim(0.0,90.0)
fig.savefig(cwd_PYTHON+'/../Figures/Angles/AvgVel_ThetaBW_V_Zoom.png')
fig.clf()
plt.close()
print('Plotting Avg Vel and Theta_BW together is complete')

Plotting Avg Vel and Theta_BW together is complete
